In [5]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### LM2FC

In [6]:
k = 4
s = -1/(k-1)*20
C_num = 3

In [16]:
my_x[0,]

array([ 0.77579936, -3.97356112,  0.49908136, -0.4876665 , -0.04236055,
       -0.14583374,  0.54      ,  0.1       ,  0.24      ])

In [9]:
file_data = 'timit_data_train_0211.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_x = my_data[:,1:len(my_data)]
n = len(my_x[:,0])
x_dim = len(my_x[0,:])

file_data = 'timit_y_train_0211.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_y = my_data[:,1] - 1 ##注意减一
my_y = my_y.reshape((len(my_y),1))

file_data = 'timit_data_test_0211.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_x_test = my_data[:,1:len(my_data)]
n_test = len(my_x_test[:,0])
x_dim = len(my_x_test[0,:])

file_data = 'timit_y_test_0211.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_y_test = my_data[:,1] - 1 ##注意减一
my_y_test = my_y_test.reshape((len(my_y_test),1))

In [12]:
my_omega = np.random.randn(x_dim,k)
my_b = np.random.randn(k)
C_num = 0.15
correct_num = 0
iter_num = 100

train_s_cr = []
test_s_cr = []

for i in range(n):
    esti = np.dot(my_x[i,:], my_omega) + my_b
    if esti.argmax() == my_y[i]:
        correct_num += 1
print("初始准确率:",correct_num/n)

for iter in range(iter_num):
    my_beta = np.zeros((n,k))
    ##### beta
    for i in range(n):
        pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
        pred_copy = pred_iter.copy()
        pred_iter[int(my_y[i])] = -float('inf')
        j = pred_iter.argmax()

        if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
            my_beta[i,j] = C_num
    
    ##### function1
    my_alpha = cp.Variable((n,k))
    obj_function = 0
    
    s1 = 0
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

            if my_y[i] != j:
                s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

        s1 += cp.sum_squares(s1_1 - s1_2)
        
    s2 = 0
    for i in range(n):
        for j2 in range(k):
            if j2 != my_y[i]:
                s2 += my_alpha[i,j2]
                
    obj_function = 0.5 * s1 - s2
    
    ##### constrains1
    constrains = []
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

            if my_y[i] != j:
                s1_2 += (my_alpha[i,j] - my_beta[i,j])

        constrains += [(s1_1 - s1_2) == 0]

    for i in range(n):
        s1 = 0
        for j in range(k):
            if j != my_y[i]:
                s1 += my_alpha[i,j]
                constrains += [s1 >= 0]
                constrains += [s1 <= C_num]

    for i in range(n):
        for j in range(k):
            if j != my_y[i]:
                constrains += [my_alpha[i,j] >= 0]
                
    prob = cp.Problem(cp.Minimize(obj_function), constrains)
    prob.solve()
    
    ##### calculate omega
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

            if my_y[i] != j:
                s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

        my_omega[:,j] = s1_1 - s1_2
        
    #print(my_omega)
    
    #####function2
    my_eta = cp.Variable(n)
    my_b = cp.Variable(k)
    
    obj_function_2 = 0
    s1 = 0
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += my_beta[i,j2]

            if my_y[i] != j:
                s1_2 += my_beta[i,j2]

        s1 += (s1_1 - s1_2) * my_b[j]

    obj_function_2 = C_num * cp.sum(my_eta) + s1
    
    ##### constrains2
    
    constrains_2 = []
    f_sim = np.dot(my_x, my_omega)

    for i in range(n):
        constrains_2 += [my_eta[i] >= 0]

    for i in range(n):
        for j in range(k):
            if j != my_y[i]:
                constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

    constrains_2 += [cp.sum(my_b) == 0]
    
    prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)
    
    prob_2.solve()
    my_b = my_b.value
    #print(my_b)
    
    ### train set correct-ratio ###
    
    correct_num = 0
    esti_mat = np.zeros(n)
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        esti_mat[i] = esti.argmax()
        if esti.argmax() == my_y[i]:
            correct_num += 1
    #print(correct_num/n)
    
    cf_mat = np.zeros((k,k))
    for i in range(n):
        cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
    #print(cf_mat)
    print("iter:{},准确率:{}".format(iter, correct_num/n))
    print(cf_mat)
    train_s_cr.append(correct_num/n)
    
    
    ### test set correct-ratio ###
    correct_num = 0
    esti_mat_test = np.zeros(n_test)
    for i in range(n_test):
        esti = np.dot(my_x_test[i,:], my_omega) + my_b
        esti_mat_test[i] = esti.argmax()
        if esti.argmax() == my_y_test[i]:
            correct_num += 1
    #print(correct_num/n)

    cf_mat = np.zeros((k,k))
    for i in range(n_test):
        cf_mat[int(my_y_test[i]),int(esti_mat_test[i])] += 1
    #print(cf_mat)
    print("准确率:{}".format(correct_num/n_test))
    print(cf_mat)
    test_s_cr.append(correct_num/n_test)

初始准确率: 0.40384615384615385
iter:0,准确率:0.6666666666666666
[[18.  7. 13.  5.]
 [ 1. 52.  0.  0.]
 [ 4.  4.  2. 10.]
 [ 5.  0.  3. 32.]]
准确率:0.5882352941176471
[[10.  2.  4.  4.]
 [ 0. 21.  0.  0.]
 [ 2.  4.  1. 11.]
 [ 5.  1.  2. 18.]]
iter:1,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:2,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:3,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:4,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  

iter:40,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:41,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:42,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:43,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:44,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]

iter:80,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:81,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:82,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]
iter:83,准确率:0.6987179487179487
[[20.  6. 14.  3.]
 [ 1. 52.  0.  0.]
 [ 2.  4.  4. 10.]
 [ 5.  1.  1. 33.]]
准确率:0.5882352941176471
[[ 9.  2.  6.  3.]
 [ 0. 21.  0.  0.]
 [ 4.  3.  1. 10.]
 [ 5.  1.  1. 19.]]


KeyboardInterrupt: 

### PWC

In [13]:
def m_distance(x1,x2,cov_m):
    return( np.dot(np.dot( (x1 - x2).T, np.linalg.inv(cov_m) ), (x1 - x2)) )

In [14]:
for label in range(1):
    
    file_data = 'timit_data_train_0211.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data)]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = 'timit_y_train_0211.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))
    
    file_data = 'timit_data_test_0211.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data)]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = 'timit_y_test_0211.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    

    label_sum = np.zeros((k,x_dim))
    label_num = np.zeros(k)
    label_cov = np.zeros((k,x_dim,x_dim)) ###
    
    for i in range(n):
        i_label = my_y[i,0]
        label_num[i_label] += 1
        label_sum[i_label,:] += my_x[i,:]
    
    label_mean = np.zeros((k,x_dim))
    for i in range(k):
        label_mean[i,:] = label_sum[i,:]/label_num[i]
        label_cov[i,:] = np.cov( ((my_x[np.where(my_y[:,0] == i),:])[0,:]) , rowvar=False) ###
        
    
    correct_num = 0
    esti_mat = np.zeros(n)
    for i in range(n):
        esti = np.zeros(k)
        for j in range(k):
            esti[j] = m_distance(my_x[i,:],label_mean[j,:], label_cov[j,:]) - np.log(label_num[j]) + 0.5*np.log(np.linalg.det(label_cov[j,:]))###
        esti_mat[i] = esti.argmin()
        if esti.argmin() == my_y[i]:
            correct_num += 1
    print(correct_num/n)
    

    correct_num = 0
    esti_mat_test = np.zeros(n_test)
    for i in range(n_test):
        esti = np.zeros(k)
        for j in range(k):
            esti[j] = m_distance(my_x_test[i,:],label_mean[j,:], label_cov[j,:]) - np.log(label_num[j]) + 0.5*np.log(np.linalg.det(label_cov[j,:])) ###

        esti_mat_test[i] = esti.argmin()
        if esti.argmin() == my_y_test[i]:
            correct_num += 1

    cf_mat = np.zeros((k,k))
    for i in range(n_test):
        cf_mat[int(my_y_test[i]),int(esti_mat_test[i])] += 1
        
    print(correct_num/n_test)
    print(cf_mat)


0.8076923076923077
0.6
[[19.  0.  0.  1.]
 [ 0. 21.  0.  0.]
 [ 9.  2.  1.  6.]
 [15.  1.  0. 10.]]
